## Model Building - Random Forest

In [17]:
import warnings
warnings.filterwarnings("ignore")

import polars as pl
import pandas as pd
import lightgbm as lgb
from glob import glob
import matplotlib.pyplot as plt
import seaborn as sns
import os
from imblearn.over_sampling import SMOTE
from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_score
from sklearn.impute import SimpleImputer
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.metrics import roc_curve, auc, roc_auc_score
from sklearn.metrics import classification_report, confusion_matrix


## Extract 10000 case_id's for num_group = 0.

In [3]:
def merge_n_case_ids(
    n_ids: int = 10000,
    data_dir: str = '/kaggle/input/cr-newagg/new_aggs',
    path_to_base: str = '/kaggle/input/datasets-cr/train_base.parquet',
    use_0: bool = True,
    as_pandas: bool = True,
    random_state: int = 28
) -> pl.DataFrame | pd.DataFrame:
    '''
    Function to merge selected case_id from parquet files, returns subset case_id from the merged dataset.

    Parameters
    ----------
    n_ids : Number of case_ids to sample initially and return (int)
    data_dir : Path to processed parquet files directory (str)
    path_to_base : Path to base file (str)
    use_0 : Use num_group1 == 0 (bool)
    as_pandas : Return as pandas DataFrame
    random_seed : Random seed (int)
    '''
    # Read the base dataframe and sample case_ids
    if as_pandas:
        base_df = pd.read_parquet(path_to_base)
        case_ids = base_df['case_id'].sample(n=n_ids, replace=False, random_state=random_state).tolist()
    else:
        base_df = pl.read_parquet(path_to_base)
        case_ids = base_df['case_id'].sample(n=n_ids, replace=False, seed=random_state).to_list()

    # Define the file pattern for fetching files
    file_pattern = '*grouped_0.parquet' if use_0 else '*grouped_rest.parquet'
    file_paths = glob(data_dir + '/' + file_pattern)

    # Initialize the merged DataFrame filtering the base DataFrame
    if as_pandas:
        df = base_df[base_df['case_id'].isin(case_ids)]
    else:
        df = base_df.filter(pl.col('case_id').is_in(case_ids))

    # Merge DataFrames with only the selected case_ids
    for path in file_paths:
        if as_pandas:
            temp = pd.read_parquet(path)
            temp = temp[temp['case_id'].isin(case_ids)]
            df = pd.merge(df, temp, on='case_id', how='outer')
        else:
            temp = pl.read_parquet(path)
            temp = temp.filter(pl.col('case_id').is_in(case_ids))
            df = df.join(temp, on='case_id', how='outer')

    # Convert to pandas if required and using Polars
    if as_pandas and isinstance(df, pl.DataFrame):
        df = df.to_pandas()

    return df

In [4]:
df = merge_n_case_ids(
    n_ids=10000,
    data_dir='/kaggle/input/cr-newagg/new_aggs',
    path_to_base='/kaggle/input/datasets-cr/train_base.parquet',
    use_0=True,
    as_pandas=True,
    random_state=28  
)

In [5]:
df

,case_id,date_decision,MONTH,WEEK_NUM,target,amount_4527230A_min,amount_4527230A_max,amount_4527230A_mean,amount_4527230A_median,amount_4527230A_sum,...,last30dayturnover_651A_sum,openingdate_857D_min,openingdate_857D_max,openingdate_857D_distinct,openingdate_857D_min_year,openingdate_857D_min_month,openingdate_857D_min_day,openingdate_857D_max_year,openingdate_857D_max_month,openingdate_857D_max_day
0,4,2019-01-04,201901,0,1,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,134,2019-01-04,201901,0,0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,199,2019-01-04,201901,0,0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,496,2019-01-08,201901,1,0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,672,2019-01-09,201901,1,0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,2702859,2020-10-02,202010,91,0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9996,2703191,2020-10-05,202010,91,0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9997,2703347,2020-10-05,202010,91,0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9998,2703348,2020-10-05,202010,91,0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Columns: 1424 entries, case_id to openingdate_857D_max_day
dtypes: float64(1230), int32(2), int64(4), int8(99), object(81), uint32(8)
memory usage: 101.7+ MB


In [8]:
# Get columns with data type 'object'
object_cols = df.select_dtypes(include=['object']).columns.tolist()

# Print the list of object columns
print("Object type columns:", object_cols)

Object type columns: ['date_decision', 'recorddate_4527225D_min', 'recorddate_4527225D_max', 'dateofcredend_289D_min', 'dateofcredend_353D_min', 'dateofcredstart_181D_min', 'dateofcredstart_739D_min', 'dateofrealrepmt_138D_min', 'lastupdate_1112D_min', 'lastupdate_388D_min', 'numberofoverdueinstlmaxdat_148D_min', 'numberofoverdueinstlmaxdat_641D_min', 'overdueamountmax2date_1002D_min', 'overdueamountmax2date_1142D_min', 'refreshdate_3813885D_min', 'dateofcredend_289D_max', 'dateofcredend_353D_max', 'dateofcredstart_181D_max', 'dateofcredstart_739D_max', 'dateofrealrepmt_138D_max', 'lastupdate_1112D_max', 'lastupdate_388D_max', 'numberofoverdueinstlmaxdat_148D_max', 'numberofoverdueinstlmaxdat_641D_max', 'overdueamountmax2date_1002D_max', 'overdueamountmax2date_1142D_max', 'refreshdate_3813885D_max', 'contractenddate_991D_min', 'openingdate_313D_min', 'contractenddate_991D_max', 'openingdate_313D_max', 'pmts_date_1107D_min', 'pmts_date_1107D_max', 'isbidproduct_390L_min', 'isdebitcard_5

In [9]:
object_cols = ['recorddate_4527225D_min', 'recorddate_4527225D_max', 'dateofcredend_289D_min', 'dateofcredend_353D_min', 'dateofcredstart_181D_min', 'dateofcredstart_739D_min', 
               'dateofrealrepmt_138D_min', 'lastupdate_1112D_min', 
               'lastupdate_388D_min', 'numberofoverdueinstlmaxdat_148D_min', 'numberofoverdueinstlmaxdat_641D_min', 'overdueamountmax2date_1002D_min', 'overdueamountmax2date_1142D_min', 'refreshdate_3813885D_min', 
               'dateofcredend_289D_max', 'dateofcredend_353D_max', 'dateofcredstart_181D_max', 'dateofcredstart_739D_max', 'dateofrealrepmt_138D_max', 'lastupdate_1112D_max', 'lastupdate_388D_max', 
               'numberofoverdueinstlmaxdat_148D_max', 'numberofoverdueinstlmaxdat_641D_max', 'overdueamountmax2date_1002D_max', 'overdueamountmax2date_1142D_max', 'refreshdate_3813885D_max', 
               'contractenddate_991D_min', 'openingdate_313D_min', 
               'contractenddate_991D_max', 'openingdate_313D_max', 'pmts_date_1107D_min', 'pmts_date_1107D_max', 'approvaldate_319D_min', 'creationdate_885D_min', 
               'dateactivated_425D_min', 'dtlastpmt_581D_min', 'dtlastpmtallstes_3545839D_min', 'employedfrom_700D_min', 'firstnonzeroinstldate_307D_min', 'approvaldate_319D_max', 
               'creationdate_885D_max', 'dateactivated_425D_max', 'dtlastpmt_581D_max', 'dtlastpmtallstes_3545839D_max', 'employedfrom_700D_max', 'firstnonzeroinstldate_307D_max', 
               'empls_employedfrom_796D_min', 'empls_employedfrom_796D_max', 'processingdate_168D_min', 'processingdate_168D_max', 'contractdate_551D_min', 'contractmaturitydate_151D_min', 
               'lastupdate_260D_min', 'contractdate_551D_max', 'contractmaturitydate_151D_max', 'lastupdate_260D_max', 'birth_259D_min', 'birthdate_87D_min', 'empl_employedfrom_271D_min', 
               'birth_259D_max', 'birthdate_87D_max', 'empl_employedfrom_271D_max', 'deductiondate_4917603D_min', 'deductiondate_4917603D_max', 'openingdate_857D_min', 'openingdate_857D_max']

# Drop the object columns from the DataFrame
df.drop(columns=object_cols, inplace=True)

# Print the new DataFrame information to confirm columns are deleted
print(df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Columns: 1358 entries, case_id to openingdate_857D_max_day
dtypes: float64(1230), int32(2), int64(4), int8(99), object(15), uint32(8)
memory usage: 96.6+ MB
None


In [10]:
print(df.columns.tolist())

['case_id', 'date_decision', 'MONTH', 'WEEK_NUM', 'target', 'amount_4527230A_min', 'amount_4527230A_max', 'amount_4527230A_mean', 'amount_4527230A_median', 'amount_4527230A_sum', 'recorddate_4527225D_distinct', 'recorddate_4527225D_min_year', 'recorddate_4527225D_min_month', 'recorddate_4527225D_min_day', 'recorddate_4527225D_max_year', 'recorddate_4527225D_max_month', 'recorddate_4527225D_max_day', 'name_4527232M_freq', 'name_4527232M_binary_0', 'name_4527232M_binary_1', 'name_4527232M_binary_2', 'name_4527232M_binary_3', 'name_4527232M_binary_4', 'name_4527232M_binary_5', 'name_4527232M_binary_6', 'name_4527232M_binary_7', 'name_4527232M_binary_8', 'name_4527232M_binary_9', 'name_4527232M_binary_10', 'name_4527232M_binary_11', 'name_4527232M_binary_12', 'name_4527232M_binary_13', 'name_4527232M_binary_14', 'name_4527232M_binary_15', 'name_4527232M_binary_16', 'annualeffectiverate_199L_min', 'annualeffectiverate_63L_min', 'contractsum_5085717L_min', 'credlmt_230A_min', 'credlmt_935A_m

In [11]:
# Convert 'date_decision' from object (string) to 'datetime64[ns]'
df['date_decision'] = pd.to_datetime(df['date_decision'])

# Extract day, month, and year into separate columns with specific names
df['date_decision_year'] = df['date_decision'].dt.year
df['date_decision_month'] = df['date_decision'].dt.month
df['date_decision_day'] = df['date_decision'].dt.day

# Display the DataFrame to see the changes
print(df)

      case_id date_decision   MONTH  WEEK_NUM  target  amount_4527230A_min  \
0           4    2019-01-04  201901         0       1                  NaN   
1         134    2019-01-04  201901         0       0                  NaN   
2         199    2019-01-04  201901         0       0                  NaN   
3         496    2019-01-08  201901         1       0                  NaN   
4         672    2019-01-09  201901         1       0                  NaN   
...       ...           ...     ...       ...     ...                  ...   
9995  2702859    2020-10-02  202010        91       0                  NaN   
9996  2703191    2020-10-05  202010        91       0                  NaN   
9997  2703347    2020-10-05  202010        91       0                  NaN   
9998  2703348    2020-10-05  202010        91       0                  NaN   
9999  2703350    2020-10-05  202010        91       0                  NaN   

      amount_4527230A_max  amount_4527230A_mean  amount_4527230

In [12]:
df.drop(columns='date_decision', inplace=True)


In [13]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Columns: 1360 entries, case_id to date_decision_day
dtypes: float64(1230), int32(5), int64(4), int8(99), object(14), uint32(8)
memory usage: 96.7+ MB


# Model Building: Random Forest

In [14]:
# Count the occurrences of each unique value in 'target'
value_counts = df['target'].value_counts().reset_index()
value_counts.columns = ['target', 'count']

# Calculate the total number of rows in the DataFrame
total_count = df.shape[0]  # This returns the number of rows

# Calculate the proportion of each 'target' value
value_counts['proportion'] = value_counts['count'] / total_count

# Print the result
print(value_counts)

   target  count  proportion
0       0   9667      0.9667
1       1    333      0.0333


The ratio of fraud to non-fraud observations is very low. Need to use techniques such as oversampling the minority class, undersampling the majority class, or using synthetic data generation methods like SMOTE (Synthetic Minority Over-sampling Technique) to balance the classes before model training.

In [15]:
# Separate features and target
X = df.drop('target', axis=1)
y = df['target'] 

# You can check the content of X and y to confirm
print(X.head()) 
print(y.head())

   case_id   MONTH  WEEK_NUM  amount_4527230A_min  amount_4527230A_max  \
0        4  201901         0                  NaN                  NaN   
1      134  201901         0                  NaN                  NaN   
2      199  201901         0                  NaN                  NaN   
3      496  201901         1                  NaN                  NaN   
4      672  201901         1                  NaN                  NaN   

   amount_4527230A_mean  amount_4527230A_median  amount_4527230A_sum  \
0                   NaN                     NaN                  NaN   
1                   NaN                     NaN                  NaN   
2                   NaN                     NaN                  NaN   
3                   NaN                     NaN                  NaN   
4                   NaN                     NaN                  NaN   

   recorddate_4527225D_distinct  recorddate_4527225D_min_year  ...  \
0                           NaN                     

In [18]:
# Marking null values with a placeholder, for example, -999
# This allows the RandomForest to split on these as a separate category
X = X.fillna(-999)

# Balancing the dataset using SMOTE
smote = SMOTE(random_state=42)
X_balanced, y_balanced = smote.fit_resample(X, y)

# Splitting the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X_balanced, y_balanced, test_size=0.2, random_state=42)

# Initialize the RandomForestClassifier
clf = RandomForestClassifier(random_state=42, class_weight='balanced')

# Perform cross-validation
scores = cross_val_score(clf, X_balanced, y_balanced, cv=5)

# Train the model
clf.fit(X_train, y_train)

# Make predictions on the test set
y_pred = clf.predict(X_test)

# Output the cross-validation results
print("Cross-validation scores:", scores)
print("Mean CV Accuracy:", scores.mean())

# Output the classification report for test set
print("Classification Report:\n", classification_report(y_test, y_pred))
print("Accuracy on Test Set:", accuracy_score(y_test, y_pred))


Cross-validation scores: [0.91802431 0.99431084 0.99844841 0.97698474 0.999224  ]
Mean CV Accuracy: 0.977398459995042
Classification Report:
               precision    recall  f1-score   support

           0       0.97      1.00      0.98      1944
           1       1.00      0.97      0.98      1923

    accuracy                           0.98      3867
   macro avg       0.98      0.98      0.98      3867
weighted avg       0.98      0.98      0.98      3867

Accuracy on Test Set: 0.982415309025084


In [ ]:
param_grid = {
    'max_depth': [80, 90, 100, 110],
    'max_features' : [600, 700],
    'min_samples_leaf': [100, 200, 300],
    'min_samples_split': [150, 250, 350],
    'n_estimators': [100, 200, 300, 1000]
}